In [1]:
# Imports
import Adafruit_PCA9685
import time
import math


In [2]:
# Set the I2C bus number and PCA9685 address
# Run `i2cdetect -y -r <bus>` to find the hex address (e.g., 0x40)
I2C_BUS = 1
PCA9685_I2C_ADDRESS = 0x40


In [3]:
# Servo helpers and demo
# Helpers for angle ↔ PWM counts (assumes 50 Hz)
FREQ = 50
PERIOD_MS = 1000.0 / FREQ

# Typical servo pulse range (adjust if your servo differs)
SERVO_MIN_MS = 1.0    # ms for min angle
SERVO_MAX_MS = 2.0    # ms for max angle
ANGLE_MIN = 0
ANGLE_MAX = 180

In [4]:
# PCA9685 initialization with diagnostics
print(f"Using I2C bus {I2C_BUS}, PCA9685 address 0x{PCA9685_I2C_ADDRESS:02x}")
try:
    pwm = Adafruit_PCA9685.PCA9685(address=PCA9685_I2C_ADDRESS, busnum=I2C_BUS)
    pwm.set_pwm_freq(FREQ)  # 50 Hz is typical for servos
    print("PCA9685 initialized successfully")
except Exception as e:
    print("Failed to initialize PCA9685:", e)
    raise


Using I2C bus 1, PCA9685 address 0x40
PCA9685 initialized successfully


In [5]:
# Convert angle in degrees to PCA9685 counts (0-4095)
def angle_to_counts(angle, min_angle=ANGLE_MIN, max_angle=ANGLE_MAX,
                    min_ms=SERVO_MIN_MS, max_ms=SERVO_MAX_MS, freq=FREQ):
    angle = max(min_angle, min(max_angle, angle))
    pulse_ms = min_ms + (angle - min_angle) / (max_angle - min_angle) * (max_ms - min_ms)
    return int(pulse_ms / (1000.0 / freq) * 4096)


In [ ]:
class ServoController:
    def __init__(self, pwm, channel, center=90):
        self.pwm = pwm
        self.channel = channel
        self.current_angle = 90  # Assume starting at center
        self.pwm.set_pwm(channel, 0, angle_to_counts(self.current_angle))
        self.center: int = center

    def move_to_center(self):
        # Ensure servo at center first
        self.pwm.set_pwm(self.channel, 0, angle_to_counts(self.center))
    def move_to(self, target_angle, speed_deg_per_sec=60.0, step_dt=0.02):
        start = self.current_angle
        delta = target_angle - start
        duration = abs(delta) / float(speed_deg_per_sec)
        if duration <= 0:
            return
        steps = max(1, int(duration / step_dt))
        dt = duration / steps
        for i in range(1, steps + 1):
            a = start + delta * (i / steps)
            self.pwm.set_pwm(self.channel, 0, angle_to_counts(a))
            time.sleep(dt)
        self.current_angle = target_angle


In [9]:
# Demo: rotate CW 60° in 1s, wait 1s, then return in 1s
channel: int = 0
center: int = 90
angle_shift: int = 60
speed_deg_per_sec: float = 60.0  # degrees per second

In [ ]:
servo_controller = ServoController(pwm, channel)

In [16]:
print("Moving CW 60° at 60°/s")
move_to(channel, center + angle_shift, speed_deg_per_sec=speed_deg_per_sec)


Moving CW 60° at 60°/s


In [ ]:
move_to(channel, center, speed_deg_per_sec=speed_deg_per_sec)


In [ ]:
# angle_to_counts(center)

In [ ]:
# while True:
pwm.set_pwm(0, 0, 150)
time.sleep(1)
pwm.set_pwm(0, 0, 650)
time.sleep(1)